In [1]:
import cv2
import numpy as np

In [2]:
def try_func(func, *args, **kwargs):
    try:
        return func(*args, **kwargs)
    except Exception as e:
        print(e)
        return None

In [3]:
import cmapy
cmap = cmapy.cmap('nipy_spectral')

In [14]:
net = cv2.dnn.readNetFromONNX('../models/handmotion/best.onnx')

In [11]:
CLASSES = ["Both_long", "One", "Both_short", "NONE"]
colors = np.random.uniform(0, 255, size=(len(CLASSES), 3))

In [12]:
def draw_bounding_box(img, class_id, confidence, x, y, x_plus_w, y_plus_h):
    label = f'{CLASSES[class_id]} ({confidence:.2f})'
    color = colors[class_id]
    cv2.rectangle(img, (x, y), (x_plus_w, y_plus_h), color, 2)
    cv2.putText(img, label, (x - 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)


In [15]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [23]:
cap = cv2.VideoCapture(0)
fps = cap.get(cv2.CAP_PROP_FPS) # 프레임 수 구하기
delay = int(1000/fps)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

history = np.zeros((height, width), dtype=np.uint8)

# 배경 제거 객체
fgbg = cv2.createBackgroundSubtractorMOG2()

while cap.isOpened():
    ret, frame = cap.read()
    if ret == False:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    
    # 배경 제거 마스크 계산
    fgmask = fgbg.apply(frame)
    
    # make 24 frame of history based on fgmask
    history = np.where(fgmask == 255, 250, history)
    history = np.where(history > 0, history - 10, history)
    
    # make clip for safety
    history = np.clip(history, 0, 255)
    
    # color map
    color_map_history = cv2.applyColorMap(history, cmap)
    
    box_img = color_map_history.copy()
    
    # detect
    length = max((height, width))
    scale = length / 640

    blob = cv2.dnn.blobFromImage(box_img, scalefactor=1 / 255, size=(640, 640))
    net.setInput(blob)
    outputs = net.forward()
    
    outputs = np.array([cv2.transpose(outputs[0])])
    rows = outputs.shape[1]
    
    boxes = []
    scores = []
    class_ids = []
    for i in range(rows):
        classes_scores = outputs[0][i][4:]
        (minScore, maxScore, minClassLoc, (x, maxClassIndex)) = cv2.minMaxLoc(classes_scores)
        if maxScore >= 0.25:
            box = [
                outputs[0][i][0] - (0.5 * outputs[0][i][2]), outputs[0][i][1] - (0.5 * outputs[0][i][3]),
                outputs[0][i][2], outputs[0][i][3]]
            boxes.append(box)
            scores.append(maxScore)
            class_ids.append(maxClassIndex)

    result_boxes = cv2.dnn.NMSBoxes(boxes, scores, 0.25, 0.45, 0.5)
    
    detections = []
    for i in range(len(result_boxes)):
        index = result_boxes[i]
        box = boxes[index]
        detection = {
            'class_id': class_ids[index],
            'class_name': CLASSES[class_ids[index]],
            'confidence': scores[index],
            'box': box,
            'scale': scale}
        detections.append(detection)
        draw_bounding_box(box_img, class_ids[index], scores[index], round(box[0] * scale), round(box[1] * scale),
                          round((box[0] + box[2]) * scale), round((box[1] + box[3]) * scale))

    cv2.imshow('frame',frame)
    cv2.imshow('bgsub',fgmask)
    cv2.imshow('history', history)
    cv2.imshow('cmaped', color_map_history)
    cv2.imshow('box', box_img)

    if cv2.waitKey(1) & 0xff == 27:
        break

try_func(cap.release)
try_func(cv2.destroyAllWindows)


print("Goodbye User")

IndexError: list index out of range

In [ ]:
try_func(cap.release)
try_func(cv2.destroyAllWindows)